# Failure Classification

## Transaction Failure
- **logical errors**: transaction cannot complete due to some internal error condiction (e.g., division by zero, violation of referential integrity)
- **system errors**: the database system must terminate an active transaction due to an error condition (e.g., deadlock)

## System Crash
- a power failure or other hardware or software failure causes the entire system to crash, and possibly reboot later
- **fail-stop assumption**: non-volatile storage contents are not corrupted by a system crash

## Disk Failure
- a head crash or other hardware failure destroys all or part of a disk, destruction is assumed to be detectable, disk drives use checksums to detect failures



# Challenges in Recovery

- Achieveing atomicity and durability *simulataneously* is difficult because modifications to A and B may correspond to separate I/O operations
    - a failure may occur after one of the modifications has been made but before both of them are made
    - modifying the database without ensuring that the transaction will commit may leave the database in an inconsistent state
    - not modifying the database may result in lost updates if failure occurs just after the transaction commits

# Two Parts of Recovery Algorithms

1. Actions taken during normal transaction processing to ensure that sufficient housekeeping information exists to recover from failures
2. Actions taken after a failure to recover the database contents to a state that ensures ACID properties

# Storage Structure

- **volatile storage**: does not survive system crashes, e.g., main memory, cache memory
- **non-volatile storage**: survives system crashes, e.g., hard disk, SSD, may still fail and lose data
- **stable storage**: a *hypothetical* form of storage that survives all failures, approximated in reality by maintaining multiple copies of data on distinct non-volatile media (e.g., using RAID)


# Data Access

- **physical blocks**: blocks residing on the disk
- **buffer blocks**: blocks residing temporarily in main memory (i.e., buffered in the buffer pool)
- the buffer pool manager moves blocks between disk and main memory using the following two operations:
    - **output**$(B)$ transfers the buffer block $B$ from main memory to disk, overwriting the corresponding physical block on disk
    - **input**$(B)$ transfers the physical block $B$ from disk to main memory (if necessary, free space in the buffer pool can be created first by evicting an existing buffer block and writing it to disk)



<img src="img/Snip20191202_145.png" width=80%/>


- each transaction $T_i$ has a private **work area** in which it keeps local copies of all data items it accesses and updates
    - $T_i's$ local copy of a data item $X$ is called $X_i$
- transferring data items between buffer blocks and the private work area done using two operations
    - **read**$(X)$ assigns the value of data item $X$ to the local variable $X_i$
    - **write**$(X)$ assigns the value of local variable $X_i$ to data item $X$ in the buffer block
    
    
    
- **output**$(B)$ need not immediately follow **write**$(X)$, the buffer pool manager can delay the **output** operation to improve I/O performance



- transaction behavior
    - must **read**$(X)$ before accessing $X$ for the first time (subsequent reads can be from local copy in work area)
    - may execute **write**$(X)$ at *any tme before the transaction commits*

# Recovery and Atomicity

- to ensure atomicity despite failures, we first output information describing the modifications to stable stoage without modifying the database itself


- log-based recovery mechanisms: **write-ahead logging (WAL)** and **redo and undo operations**
    - real-world databases use more elaborate log-based recovery algorithms such as **ARIES** (Algorithms for Recovery and Isolation Exploiting Semantics)



# Log-Based Recovery

- a **recovery log** is a sequential (i.e., append-only) structure recorded in stable storage: it comprises a sequence of **log records** and represents a history of updates to the database


- when a transaction $T_i$ starts, a log record <$T_i$ **start**> is written
- before $T_i$ executes **write($X$)**, a log record of the form <$T_i$, $X$, $V_1$, $V_2$> is written, where $V_1$ is the value of $X$ before the write (the **old value**) and $V_2$ is the value to be written to $X$ (the **new value**)
- when $T_i$ finishes its last statement, the log record <$T_i$ **commit**> is written


- a transaction is said to have committed when its comit log record (and all prior log records) is output to stable storage


# Immediate vs Deferred Modification

- despite all updates persist in the recovery log, we still need to update the state of the database on disk (modifying table rows and index entries) -> allows us to answer queries efficiently without having to replay the recovery log


- the **immediate-modification** scheme allows updates of an uncommitted transaction to be made to the buffer, or the disk itself, before the transaction commits
    - **write-ahead logging (WAL) rule**: update log record must be output to stable storage *before* database item is written to disk
    - output of updated blocks to disk can take place at *any time* before or after transaction commit (not always immediately)
    - order in which blocks are output to disk can be different from the order in which they are written to (buffer pool)


<img src="img/Snip20191203_146.png" width=80%/>


- the **deferred-modification** scheme writes to buffer/disk only at the time of transaction commit
    - this simplifies some aspects of recovery but imposes the overhead of storing a local copy
    

# Undo & Redo Operations

- on recovery, any transactions that did not finish prior to the failure (i.e., did not commit or abort) must be cleaned up; this involves undoing/redoing entire transactions as well as individual log records


- for log records:
    - **undo** of <$T_i$, $X$, $V_1$, $V_2$> executes **write($X$)** with value $V_1$
    - **redo** of <$T_i$, $X$, $V_1$, $V_2$> executes **write($X$)** with value $V_2$
    

- for transactions
    - **undo** of $T_i$ undoes all the log records corresponding to updates applied by $T_i$ by going *backward* from the last log record of $T_i$
        - each time a log record <$T_i$, $X$, $V_1$, $V_2$> is undone, a special **compensation log record** <$T_i$, $X$, $V_1$> is appended to the log
        - the procedure ends bya appending <$T_i$ **abort**> to the log
    - **redo** of $T_i$ redoes all the log records corresponding to the updates applied by $T_i$ (including any redo log records) by going *forward* from the first log record of $T_i$
        - no additional logging


# Undo / Redo on Recovering from Failure

- Transaction $T_i$ needs to be **undone** if the log
    - contains the record <$T_i$ **start**>
    - but does not contain either <$T_i$ **commit**> or <$T_i$ **abort**>
- Transaction $T_i$ needs to be **redone** if the log
    - contains the record <$T_i$ **start**>
    - and contains the record <$T_i$ **commit**> or <$T_i$ **abort**>


- if $T_i$ is undone it will end up in the **aborted** state, otherwise if $T_i$ is redone it may end up in the **committed or aborted** state


- if $T_i$ was undone and the <$T_i$ **abort**> record wriiten to the log prior to the failure, then on recovery $T_i$ may be redone anyway including all the original actions as well as any compensation log records; this is known as **repeating history**


<img src="img/Snip20191203_147.png" width=80%/>

# Checkpoints

- redoing/undoing all transactions recorded in the recovery log can be slow
    1. processing the entire log is time-consuming
    2. we might unnecessarily redo transactions that have already finished and output their updates to the database


- real world databases streamline the recovery procedure by periodically saving a checkpoint
    1. if any log records are buffered in main memory then output them to stable storage
    2. Then output any remaining modified buffer blocks to the disk
    3. Finally write a log record <**checkpoint** $L$> to stable storage where $L$ is a list of all transactions active at the time of checkpoint
    

- all updates must be stopped while creating the checkpoint which is very disruptive


- during recovery, we process the log roughly from the most recent checkpoint
    - scan backward from the end of log to find the most recent <**checkpoint** $L$> record
    - only transactions that are in $L$ or started after the checkpoint need to be redone or undone
    - transactions that committed or aborted before the checkpoint already have all their updates output to disk
    

- some earlier part of the log may also be needed for undo operations
    - during the undo phase, continue scanning backwards until a record <$T_i$ **start**> is found for every transaction $T_i$ in $L$
    - parts of the recovery log prior to earliest <$T_i$ **start**> record above are not needed for recovery, and can be discarded
    
    

<img src="img/Snip20191203_148.png" width=80%/>

# Fuzzy Checkpointing

- to avoid long interruption of normal processing during checkpointing, allow updates to occur during checkpointing


- fuzzy checkpointing
    1. temporarily stop all updates by transactions
    2. write a <**checkpoint** $L$> log record and force log to stable storage
    3. note list $M$ of modified buffer blocks
    4. resume ordinary transaction execution
    5. output to disk all modified buffer blocks in list $M$
        - blocks should not be updated while being output
        - following WAL rule: all log records pertaining to a block must be output before the block is output
    6. store a pointer to the **checkpoint** record in a fixed position **last_checkpoint** on disk
    
    
<img src="img/Snip20191203_149.png" width=80%/>

- when recovering using a fuzzy checkpoint, start scan from the **checkpoint** record pointed to by the **last_checkpoint**
    - log records before **last_checkpoint** have their updates reflected in database on disk, and need not be redone
    - incomplete checkpoints, where system had crashed while performing checkpoint, do not break ACID properties

# Log Record Buffering

- LRB: log records are buffered in main memory, instead of being output directly to stable storage
- log records are output to stable storage when a block of log records in the buffer is full, or a **log force** operation is executed
- log force is performed to commit a transaction by forcing all its log records (including the commit record) to stable storage
- several log records can be output using a single output operation, reducing the I/O cost


- rules
    - log records are output to stable storage in the order in which they are created
    - transaction $T_i$ enters the commit state only when the log record <$T_i$ **commit**> has been output to stable storage
    - we must follow a more general form of the **write-ahead logging** (WAL) rule: before a block of data in main memory is output to the disk, all log records pertaining to data in that block must have been output to stable storage

# Failure of Nonvolatile Storage

- periodically **dump** the entire content of the database to stable storage; no transaction may be active during the dump procedure. A procedure similar to non-fuzzy checkpointing must take place
    1. output all log records currently residing in main memory to stable storage
    2. output all modified buffer blocks to disk
    3. copy the contents of the entire database to stable storage
    4. output a record <**dump**> to recovery log on stable storage


# Simplified Recovery Algorithm: Complete Details and Example



## Recovery Algorithm

- **logging** (during normal operation)
    - append <$T_i$ **start**> at transaction start
    - append <$T_i$, $X$, $V_1$, $V_2$> for each update from $V_1$ to $V_2$
    - append <$T_i$ **commit**> at transaction end


- **transaction rollback** (during normal operation)
    - let $T_i$ be the transaction to be rolled back
    - scan log backwards from the end, and for each log record of $T_i$ of the form <$T_i$, $X_j$, $V_1$, $V_2$>
        - perform the undo by applying **write**($X_j$) with the value $V_1$
        - append a compensation log record <$T_i$, $X_j$, $V_1$>
    - once the record <$T_i$ **start**> is found stop the scan and append the log record <$T_i$ **abort**>


- **recovery from failure** (two phases)
    - **redo phase**: replay updates of all transactions since the last checkpoint, regardless of their outcome
    - **undo phase**: undo all incomplete transactions


- redo phase
    1. find last <**checkpoint** $L$> record, set `undo-list := L`
    2. scan forward from the checkpoint record
        - whenever a record <$T_i$, $X_j$, $V_1$, $V_2$> is found, redo it by applying **write**($X_j$) with value $V_2$
        - whenever a compensation record <$T_i$, $X_j$, $V_1$> is found, redo it by applying **write**($X_j$) with value $V_1$
        - whenever a log record <$T_i$ **start**> is found, add $T_i$ to `undo-list`
        - whenever a log record <$T_i$ **commit**> or <$T_i$ **abort**> is found, remove $T_i$ from `undo-list`


- undo phase
    - scan log backwards from the end
        - whenever a log record <$T_i$, $X_j$, $V_1$, $V_2$> is found where $T_i$ is in `undo-list` perform same actions as for transaction rollback
            - perform undo by applying **write**($X_j$) with value $V_1$
            - append a compensation log record <$T_i$, $X_j$, $V_1$>
        - whenever a log record <$T_i$ **start**> is found where $T_i$ is in `undo-list`
            - append a log record <$T_i$ **abort**>
            - remove $T_i$ from `undo-list`
        - stop when `undo-list` is empty


- resume ordinary transaction processing after the undo phase ends


<img src="img/Snip20191203_150.png"/>